<img src=https://static.wixstatic.com/media/f6bd9a_6a972d9b31324653bf198c38ce94339e~mv2.png>

# Carrusel de RDBMS
## Ejercicios de SQL resueltos

En este cuaderno resuelvo los ejercicios del 1 al 10 del documento: 'Desarrollo de Aplicaciones en Entornos de Cuarta Generación y Herramientas CASE: Ejercicios de SQL.'

Para ello utilizo una version de la base de datos -que nombre "practicas_hospital"- modificada para ser compatible en esta ocasion, para **MySQL**, siguiendo la estructura del libro.


Tanto el diagrama de **Entidad-Relacion**, como el **Script** para cargar la base de datos (que permite, se pueda replicar el ejercicio) se encuentran disponibles en el mismo repositorio.

#### Celda para configurar el entorno



In [ ]:
# !pip install mysql-connector-python
# !pip install pandas
# !pip install sqlalchemy

#### Importamos librerias y se realiza la conexion utilizando un archivo json con la configuracion local de la base de datos

Si se desea replicar lo propio usando el script y este cuaderno, basta con crear el archivo json con el par de valores que contengan su informacion local y cargarlo al directorio.

host:

user:

password:

database:


In [1]:
# importar las librerias
import json
from sqlalchemy import create_engine
import pandas as pd

# Cargar los detalles de la conexión desde un archivo de configuración propio
with open('config.json', 'r') as f:
    config = json.load(f)

# Crear la cadena de conexión
connection_string = f"mysql+mysqlconnector://{config['user']}:{config['password']}@{config['host']}/{config['database']}"

# Crear la conexión
engine = create_engine(connection_string)

# Abrir una conexión
conn = engine.connect()


#### Realizamos una prueba y se Inspecciona la base de datos "practicas_hospital"

In [2]:
# Definir consulta en SQL compatible con MySQL
try:
    QUERY = """
    SHOW TABLES;
    """
# Ejecutar la consulta y obtener los resultados en un DataFrame
    df = pd.read_sql_query(QUERY, conn)
# Mostrar los primeros resultados
    print("La conexión se ha realizado con éxito.\n")
    print(df)
except Exception as e:
    print("Hubo un error al realizar la conexión:", e)
    

La conexión se ha realizado con éxito.

  Tables_in_practicas_hospital
0                 departamento
1                       doctor
2                     empleado
3                      enfermo
4                     hospital
5                    ocupacion
6                    plantilla
7                         sala


#### Inspeccion a las tablas de la base de datos "practicas_hospital"
#### Con la finalidad de observar su estructura y tipo de datos

In [3]:
df = df.rename(columns={'Tables_in_practicas_hospital': 'Tablas'})
for valor in df['Tablas']:
    QUERY = f'DESCRIBE {valor}'
    df1 = pd.read_sql_query(QUERY, conn)
    print(f'El contenido de la tabla {valor}\n')
    print(df1)
    print()

El contenido de la tabla departamento

     Field         Type Null  Key Default Extra
0  DEPT_NO          int   NO  PRI    None      
1  DNOMBRE  varchar(14)  YES         None      
2      LOC  varchar(14)  YES         None      

El contenido de la tabla doctor

          Field         Type Null  Key Default Extra
0  HOSPITAL_COD          int   NO         None      
1     DOCTOR_NO          int   NO  PRI    None      
2      APELLIDO  varchar(13)  YES         None      
3  ESPECIALIDAD  varchar(16)  YES         None      

El contenido de la tabla empleado

        Field         Type Null  Key Default Extra
0      EMP_NO          int   NO  PRI    None      
1    APELLIDO  varchar(10)  YES         None      
2      OFICIO  varchar(10)  YES         None      
3         DIR          int  YES         None      
4  FECHA_ALTA         date  YES         None      
5     SALARIO          int  YES         None      
6    COMISION          int  YES         None      
7     DEPT_NO          int

### Resolucion de los ejercicios numerados del 1 al 10

##### 1. Encuentre a todos los miembros del personal cuyo nombre empiece por 'H'.

In [4]:
QUERY = """
SELECT APELLIDO
FROM plantilla
WHERE APELLIDO LIKE 'H%';
"""
df = pd.read_sql_query(QUERY, conn)
df.head()

,APELLIDO
0,Higueras D.
1,Hernandez J.


##### 2. Quienes son las enfermeras y enfermeros que trabajan en turnos de Tarde o Mañana?

In [5]:
QUERY = """
SELECT APELLIDO, FUNCION, TURNO
FROM PLANTILLA
WHERE FUNCION LIKE 'ENFERMER%' AND TURNO IN ('T', 'M');
"""
df = pd.read_sql_query(QUERY, conn)
df

,APELLIDO,FUNCION,TURNO
0,Higueras D.,Enfermera,T
1,Hernandez J.,Enfermero,T
2,Diaz B.,Enfermera,T
3,Carlos R.,Enfermera,T
4,Bocina G.,Enfermero,M
5,Frank H.,Enfermera,T


##### 3. Haga un listado de las enfermeras que ganan entre 2.000.000 y 2.500.000 Pts.


In [6]:
QUERY = """
SELECT APELLIDO, FUNCION, SALARIO
FROM PLANTILLA
WHERE FUNCION IN ('ENFERMERO', 'ENFERMERA') AND (SALARIO BETWEEN 2000000 AND 2500000);
"""
df = pd.read_sql_query(QUERY, conn)
df

,APELLIDO,FUNCION,SALARIO
0,Higueras D.,Enfermera,2005000
1,Diaz B.,Enfermera,2262000
2,Carlos R.,Enfermera,2119000


##### 4. Mostrar, para todos los hospitales, el código de hospital, el nombre completo 
del hospital y su nombre abreviado de tres letras (a esto podemos llamarlo ABR).
Ordenar la recuperación por esta abreviatura.

In [7]:
QUERY = """
SELECT HOSPITAL_COD, NOMBRE, LEFT(NOMBRE, 3) AS ABR 
FROM HOSPITAL
ORDER BY ABR;
"""
df = pd.read_sql_query(QUERY, conn)
df

,HOSPITAL_COD,NOMBRE,ABR
0,18,General,Gen
1,22,La Paz,La
2,13,Provincial,Pro
3,45,San Carlos,San


##### 5. En la tabla DOCTOR otorgar a Cardiología un código de 1, a Psiquiatría un código de 2, a Pediatría un código de 
3 y a cualquier otra especialidad un código de 4. Recuperar todos los doctores, su especialidad y el código asignado.

In [8]:
QUERY = """
SELECT APELLIDO, ESPECIALIDAD,
CASE
	WHEN ESPECIALIDAD = 'Cardiología' THEN 1
    WHEN ESPECIALIDAD = 'Psiquiatría' THEN 2
	WHEN ESPECIALIDAD = 'Pediatría' THEN 3
    ELSE  4
END AS CODIGO
FROM DOCTOR
ORDER BY CODIGO;
"""
df = pd.read_sql_query(QUERY, conn)
df

,APELLIDO,ESPECIALIDAD,CODIGO
0,Lopez A.,Cardiología,1
1,Cajal R.,Cardiología,1
2,Cabeza D.,Psiquiatría,2
3,Galo D.,Pediatría,3
4,Niqo P.,Pediatría,3
5,Best K.,Urología,4
6,Adams C.,Neurología,4
7,Miller G.,Ginecología,4


##### 6. Hacer un listado de los nombres de los pacientes y la posición de la primera letra 'A' que aparezca 
en su apellido, tomando como referencia la primera letra del mismo

In [9]:
QUERY = """
SELECT APELLIDO, POSITION('A' IN UPPER(APELLIDO)) AS POSICION
FROM ENFERMO;
"""
df = pd.read_sql_query(QUERY, conn)
df

,APELLIDO,POSICION
0,Laguia M.,2
1,Serrano V.,5
2,Fernandez M,5
3,Domin S.,0
4,Neal R.,3
5,Cervantes M.,5
6,Miller G.,0
7,Ruiz P.,0
8,Fraser A.,3
9,Benitez E.,0


##### 7. Queremos conseguir:
|COMENTARIO|
|:---------:|
|El departamento de CONTABILIDAD esta en SEVILLA|
|El departamento de INVESTIGACIÓN esta en MADRID|
|El departamento de VENTAS esta en BARCELONA|
|El departamento de PRODUCCIÓN esta en BILBAO|

In [10]:
QUERY = """
SELECT CONCAT('El departamento de ',DNOMBRE,' se encuentra en  ',LOC) AS COMENTARIO 
FROM DEPARTAMENTO;
"""
df = pd.read_sql_query(QUERY, conn)
pd.set_option('display.max_colwidth', None)
df
# pd.reset_option('display.max_colwidth')

,COMENTARIO
0,El departamento de CONTABILIDAD se encuentra en MADRID
1,El departamento de INVESTIGACIÓN se encuentra en BILBAO
2,El departamento de VENTAS se encuentra en SEVILLA
3,El departamento de OPERACIONES se encuentra en MALAGA


##### 8. Para cada empleado cuyo apellido contenga una "N", queremos que nos devuelva "nnn", 
pero solo para la primera ocurrencia de la "N". La salida debe estar ordenada por apellido ascendentemente.


In [11]:
QUERY = """
SELECT CONCAT(
				LEFT(APELLIDO, POSITION('N' IN UPPER(APELLIDO)) - 1),
                'nnn',
                RIGHT(APELLIDO, CHAR_LENGTH(APELLIDO) - POSITION('N' IN UPPER(APELLIDO)))
                ) AS TRES_n
FROM EMPLEADO
WHERE POSITION('N' IN UPPER(APELLIDO)) <> 0
ORDER BY TRES_n;
"""
df = pd.read_sql_query(QUERY, conn)
df

,TRES_n
0,Alonnnso
1,Arennnas
2,Fernnnandez
3,Jimennnez
4,Jimennno
5,nnnegro
6,Sannnchez


##### 9. Para cada empleado se pide que salga su salario total (salario mas comisión) y luego su salario fragmentado,
es decir, en centenas de mil, decenas de mil... decenas y unidades. La salida debe estar ordenada por el salario 
y el apellido descendéntemente

In [12]:
QUERY = """
SELECT APELLIDO, SALARIO + IFNULL(COMISION,0) AS SALARIO_TOTAL,
		SUBSTRING(SALARIO + IFNULL(COMISION,0), 1,1) AS CM,
		SUBSTRING(SALARIO + IFNULL(COMISION,0), 2,1) AS DM,
		SUBSTRING(SALARIO + IFNULL(COMISION,0), 3,1) AS M,
		SUBSTRING(SALARIO + IFNULL(COMISION,0), 4,1) AS C,
		SUBSTRING(SALARIO + IFNULL(COMISION,0), 5,1) AS D,
		SUBSTRING(SALARIO + IFNULL(COMISION,0), 6,1) AS U
FROM EMPLEADO
ORDER BY SALARIO_TOTAL DESC, APELLIDO DESC;
"""
df = pd.read_sql_query(QUERY, conn)
df

,APELLIDO,SALARIO_TOTAL,CM,DM,M,C,D,U
0,Rey,650000,6,5,0,0,0,0
1,Gil,390000,3,9,0,0,0,0
2,Fernandez,390000,3,9,0,0,0,0
3,Jimenez,386750,3,8,6,7,5,0
4,Negro,370500,3,7,0,5,0,0
5,Arenas,344500,3,4,4,5,0,0
6,Cerezo,318500,3,1,8,5,0,0
7,Arroyo,247000,2,4,7,0,0,0
8,Sala,227500,2,2,7,5,0,0
9,Tovar,195000,1,9,5,0,0,0


##### 10. Para cada empleado que no tenga comisión o cuya comisión sea mayor que el 15% de su
salario, se pide el salario total que tiene. Este será: si tiene comisión su salario mas su comisión, 
y si no tiene, su salario mas su nueva comisión (15% del salario). La salida deberá estar ordenada por
el oficio y por el salario que le queda descendéntemente.

In [13]:
QUERY = """
SELECT APELLIDO, OFICIO, 
CASE
	WHEN IFNULL(COMISION,0) = 0 THEN SALARIO * 1.15
    ELSE SALARIO + COMISION
END AS SALARIO_TOTAL
FROM EMPLEADO
WHERE IFNULL(COMISION,0) = 0 OR COMISION > SALARIO * 0.15
ORDER BY OFICIO, 3 DESC;
"""
df = pd.read_sql_query(QUERY, conn)
df

,APELLIDO,OFICIO,SALARIO_TOTAL
0,Gil,Analista,448500.0
1,Fernandez,Analista,448500.0
2,Jimenez,Director,444762.5
3,Negro,Director,426075.0
4,Cerezo,Director,366275.0
5,Muñoz,Empleado,194350.0
6,Alonso,Empleado,164450.0
7,Jimeno,Empleado,142025.0
8,Sanchez,Empleado,119600.0
9,Rey,Presidente,747500.0


In [14]:
# Cerrar la conexión
conn.close()

# Autor
### Oscar Gutierrez Leal
15 - 03 - 2024